In [1]:
# Initialization vector (IV) is typically a public random string that is used as a third input—in addition to the key and plaintext—into the encryption algorithm
# The IV is supposed to be different each time you encrypt, preventing the same data from being encrypted to the same ciphertext
# PKCS7 padding appends n bytes, with each padding byte holding the value n: if 3 bytes of padding are needed, it appends \x03\x03\x03. Similarly, if 2 bytes of padding are needed, it appends \x02\x02.
# ANSI X.923 padding all appended bytes are 0, except for the last byte, which is the length of the total padding. In this example, 3 bytes of padding is \x00\x00\x03, and two bytes of padding is \x00\x02.
#
# If you reuse a key and IV pair, you will get predictable output for predictable headers. Parts of your messages that you might be inclined not to think about at all,
# because they are boilerplate or contain hidden structure, will become a liability; adversaries can use predictable ciphertext to learn about your keys.
#

In [13]:
# https://cryptography.io/en/latest/ pip install cryptography
# pip install cryptography

from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

def generate_ecdh_keys():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    return private_key, public_key

def derive_shared_secret(private_key, peer_public_key):
    return private_key.exchange(ec.ECDH(), peer_public_key)

def derive_key(shared_secret):
    # Derive a 128-bit key (16 bytes)
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=16,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    )
    return hkdf.derive(shared_secret)
# Encryption and Decryption:
from cryptography.hazmat.primitives.ciphers.aead import AESCCM

def encrypt_aes_ccm(key, plaintext, nonce):
    aesccm = AESCCM(key)
    return aesccm.encrypt(nonce, plaintext, None)

def decrypt_aes_ccm(key, ciphertext, nonce):
    aesccm = AESCCM(key)
    return aesccm.decrypt(nonce, ciphertext, None)

# Nonce Construction
import struct

def construct_nonce(packet_counter, direction_bit, iv):
    # Ensure packet_counter is 39 bits
    if packet_counter >= 2**39:
        raise ValueError("Packet counter must be a 39-bit integer")
    
    packet_counter_bytes = packet_counter.to_bytes(5, byteorder='big')
    # Ensure only the last 39 bits are used (5 bytes minus 1 bit)
    packet_counter_bytes = (int.from_bytes(packet_counter_bytes, byteorder='big') & ((1 << 39) - 1)).to_bytes(5, byteorder='big')
    
    direction_bit_bytes = direction_bit.to_bytes(1, byteorder='big')
    # Concatenate the components to form the nonce
    nonce = packet_counter_bytes[:-1] + direction_bit_bytes[:1] + iv
    return nonce



Ref:
[NCC Group](https://research.nccgroup.com/2021/02/17/cryptopals-exploiting-cbc-padding-oracles/)
[Poddle Attack](https://medium.com/@c0D3M/poodle-attack-explained-ed6a1cd0667d)